In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay

# Part 1

In [ ]:
# Load data
houses = pd.read_csv("data_assignment2.csv")
houses

In [ ]:
# Scatter plot living area vs prices
plt.scatter(houses[['Living_area']], houses[['Selling_price']],  c = 'b', s = 50, alpha = 0.4)
plt.title('Scatterplot')
plt.xlabel('Living area')
plt.ylabel('Selling price (million)')
plt.show()

In [ ]:
# Check for NaN
houses[['Living_area','Selling_price']].isnull().values.any()

In [ ]:
# Fit linear regression model area and price
model = LinearRegression().fit(houses[['Living_area']], houses[['Selling_price']])

In [ ]:
# Plot data and fit
xfit=np.linspace(min(houses['Living_area']),max(houses['Living_area']), 1000) #1000 evenly spaced points in [0, 55].
yfit=model.predict(xfit[:, np.newaxis])
plt.scatter(houses[['Living_area']], houses[['Selling_price']], c = 'b', s = 50, alpha = 0.4)
plt.plot(xfit, yfit)
plt.title('Regression line')
plt.xlabel('Living area')
plt.ylabel('Selling price (million)')
plt.show()

In [ ]:
# Residual plot
pred_price = model.predict(houses[['Living_area']])
residuals = pred_price - houses[['Selling_price']]
plt.scatter(pred_price, residuals, c = 'b', s = 50, alpha = 0.4)
plt.hlines(y = 0, xmin = 3400000, xmax = 6500000)
plt.title('Residual plot')
plt.ylabel('Residuals')

In [ ]:
# Extract indicies of outliers > |1 700 00|
outliers_ind = residuals[abs(residuals) > 1700000].dropna()
outliers_ind = outliers_ind.index
outliers_ind

In [ ]:
# Remove indicies from data
houses = houses.drop(houses.index[outliers_ind])
houses

In [ ]:
# New model linear regression area and price
model = LinearRegression().fit(houses[['Living_area']], houses[['Selling_price']])

In [ ]:
# Scatter plot of data and fitted model
xfit=np.linspace(min(houses['Living_area']),max(houses['Living_area']), 1000) #1000 evenly spaced points in [0, 55].
yfit=model.predict(xfit[:, np.newaxis])
plt.scatter(houses[['Living_area']], houses[['Selling_price']], c = 'b', s = 50, alpha = 0.4)
plt.plot(xfit, yfit)
plt.title('Regression line')
plt.xlabel('Living area')
plt.ylabel('Selling price (million)')
plt.show()

In [ ]:
# Slope
print(model.coef_)

# Intersection
print(model.intercept_)

In [ ]:
# Output of predictions: area 100^2, 150^2 and 200^2 
print(model.predict([[100]]))
print(model.predict([[150]]))
print(model.predict([[200]]))

In [ ]:
# Residual plot new model
pred_price = model.predict(houses[['Living_area']])
residuals = pred_price - houses[['Selling_price']]
plt.scatter(pred_price, residuals, c = 'b', s = 50, alpha = 0.4)
plt.hlines(y = 0, xmin = 3000000, xmax = 7200000)
plt.title('Residual plot')
plt.ylabel('Residuals')


# Part 2

In [ ]:
# load iris data and create dataframe
from sklearn.datasets import load_iris
iris_raw = load_iris()
iris_raw

In [ ]:
# Split into trainging and test sets
x_train, x_test, y_train, y_test = train_test_split(iris_raw.data, 
                                                    iris_raw.target, 
                                                    test_size=0.4, random_state=0)

logReg = LogisticRegression(multi_class='ovr', solver='liblinear')

# Fit logistic regression model to training data
model = logReg.fit(x_train, y_train)

# Plot confusion matrix
plot_confusion_matrix(model,x_test,y_test, cmap='Blues')


In [ ]:
k = 3
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(x_train, y_train)
#%%

y_pred = knn.predict(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))   

plot_confusion_matrix(model_knn,x_test,y_test, cmap='Blues')
